In [59]:
# import pyro
# import torch
# import torch.tensor as tensor
# import pyro.distributions as dist
# # from torch.distributions import Binomial, Gamma, Uniform
# from pyro.distributions import Binomial, Bernoulli, Categorical, Dirichlet, DirichletMultinomial, Beta, BetaBinomial, Uniform, Gamma, Multinomial

import numpy as np

import scipy
from skopt import gp_minimize 
from scipy.stats import binom as ScipyBinom
from matplotlib import pyplot
from torch import tensor

from collections import namedtuple
import time
seed = 0

In [55]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
from mvl import genData, likelihoods, bayes

In [25]:
import copy
resSim = {
        "allRes": None,
        "nEpochs": None,
        "bestRes": {
            "pis": None,
            "alphas": None,
            "PDV_c1true": None,
            "PDV_c2true": None,
            "PDV_cBothTrue": None,
            "PDV_c1inferred": None,
            "PDV_c2inferred": None,
            "PDV_cBothInferred": None,
        }
    }

cached6NormalSimRes = []

rrsSim = tensor([[2, 2, 1.5]])
pisSim = tensor([[.05, .05, .05]])

nCases = tensor([15e3, 15e3, 6e3])
nCtrls = tensor(5e5)
i = 0
for rrsSimRun in rrsSim:
    for pisSimRun in pisSim:
        afMeanRun = 1e-4
        generatingFn = genData.v6normal
        # In DSB:
        # 	No ID	ID	
        #         ASD+ADHD	684	217	
        #         ASD	3091	871	
        #         ADHD	3206	271	
        #         Control	5002	-	

        #         gnomAD	44779	(Non-Finnish Europeans in non-psychiatric exome subset)	

        #         Case total:	8340		
        #         Control total:	49781		
        # so we can use pDBoth = .1 * total_cases
        # needs tensor for shapes, otherwise "gamma_cpu not implemente for long", e.g rrShape=50.0 doesn't work...
        paramsRun = genData.genParams(rrMeans=rrsSimRun, pis=pisSimRun, afMean=afMeanRun, rrShape=tensor(50.), afShape=tensor(50.), nCases=nCases, nCtrls=nCtrls)[0]
        
        pDsRun = paramsRun["pDs"]
        pisRun = paramsRun["diseaseFractions"]
        print("params are:", paramsRun)
        
        cached6NormalSimRes.append({"params": paramsRun, "runs": []})
        for y in range(0, 10):
            start = time.time()
            r = generatingFn(**paramsRun)
            print("took", time.time() - start)
            
            resPointer = {
                **r,
                "generatingFn": generatingFn,
                "results": None,
            }

            cached6NormalSimRes[i]["runs"].append(resPointer)
            
            print(f"Run: {i}, {y}")
            
            xsRun = resPointer["altCounts"]
            afsRun = resPointer["afs"]
            affectedGenesRun = resPointer["affectedGenes"]
            unaffectedGenesRun = resPointer["unaffectedGenes"]

            runCostFnIdx = 16

            nEpochsRun = 10
            print("nEpochsRun", nEpochsRun)
            
            res = likelihoods.fitFnBivariateAnnealing(xsRun, pDsRun, nEpochs=nEpochsRun, minLLThresholdCount=20, debug=True, costFnIdx=runCostFnIdx)
            bestRes = res["params"][-1]

            inferredPis = tensor(bestRes[0:3]) # 3-vector
            inferredAlphas = tensor(bestRes[3:]) # 4-vector, idx0 is P(!D|V)

            #### Calculate actual ###
            component1Afs = afsRun[affectedGenesRun[0]]
            c1true = (component1Afs / afMeanRun).mean(0)

            component2Afs = afsRun[affectedGenesRun[1]]
            c2true = (component2Afs / afMeanRun).mean(0)

            componentBothAfs = afsRun[affectedGenesRun[2]]
            cBothTrue = (componentBothAfs / afMeanRun).mean(0)

            ### calculate inferred values
            pds = tensor([1-pDsRun.sum(), *pDsRun])
            alphas = inferredAlphas.numpy()
            c1inferred = Dirichlet(tensor([alphas[0], alphas[1], alphas[0], alphas[2]]) * pds).sample([10_000]).mean(0)
            c2inferred = Dirichlet(tensor([alphas[0], alphas[0], alphas[2], alphas[2]]) * pds).sample([10_000]).mean(0)
            cBothInferred = Dirichlet(tensor([alphas[0], (alphas[1] + alphas[3]), (alphas[2] + alphas[3]), (alphas[1] + alphas[2] + alphas[3])]) * pds).sample([10_000]).mean(0)

            print(f"\n\nrun {i} results for rrs: {rrsSimRun}, pis: {pisSimRun}")
            print("Inferred pis:", inferredPis)
            print("\nP(D|V) true ans in component 1:", c1true)
            print("P(D|V) inferred in component 1:", c1inferred)
            print("\nP(D|V) true ans in component 1:", c2true)
            print("P(D|V) inferred in component both:", c2inferred)
            print("\nP(D|V) true ans in component both:", cBothTrue)
            print("P(D|V) inferred in component both:", cBothInferred,"\n\n")

            resToStore = copy.deepcopy(resSim)
            resToStore["allRes"] = res
            resToStore["nEpochs"] = nEpochsRun
            br = resToStore["bestRes"]
            br["pis"] = inferredPis
            br["alphas"] = inferredAlphas
            br["PDV_c1true"] = c1true
            br["PDV_c2true"] = c2true
            br["PDV_cBothTrue"] = cBothTrue
            br["PDV_c1inferred"] = c1inferred
            br["PDV_c2inferred"] = c2inferred
            br["PDV_cBothInferred"] = cBothInferred

            resPointer["results"] = resToStore
        
        i += 1

pDs are: tensor([0.0280, 0.0280, 0.0112])
params are: {'nGenes': 20000, 'nCases': tensor([15000., 15000.,  6000.]), 'nCtrls': tensor(500000.), 'pDs': tensor([0.0280, 0.0280, 0.0112]), 'diseaseFractions': tensor([0.0500, 0.0500, 0.0500]), 'rrShape': tensor(50.), 'rrMeans': tensor([2.0000, 2.0000, 1.5000]), 'afShape': tensor(50.), 'afMean': 0.0001}
TESTING WITH: nCases tensor([15000., 15000.,  6000.]) nCtrls tensor(500000.) rrMeans tensor([2.0000, 2.0000, 1.5000]) rrShape tensor(50.) afMean 0.0001 afShape tensor(50.) diseaseFractions tensor([0.0500, 0.0500, 0.0500]) pDs tensor([0.0280, 0.0280, 0.0112])
tensor([[4.3519, 2.7890, 5.5548],
        [2.8917, 4.4436, 5.2706],
        [3.8678, 4.4712, 5.3107],
        ...,
        [4.7168, 4.2715, 6.7835],
        [4.6791, 3.9330, 4.3859],
        [2.6210, 1.9142, 4.4797]], dtype=torch.float64)
startIndices [0, tensor(1000.), tensor(2000.)] endIndices tensor([1000., 2000., 3000.])
totalSamples 536000
took 8.120481967926025
Run: 0, 0
nEpochsRun 1

/Users/alexkotlar/projects/tada/mvl/likelihoods.py:257: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  altCountsFlat = tensor(altCountsFlat)


best ll: 91056.366085773, bestParams: [0.021506596, 0.032269087, 0.032386456, 7456.216, 13592.753, 24910.842, 18917.502]
epoch 0
     fun: 90678.94727806201
 message: ['Maximum number of iteration reached']
    nfev: 18988
    nhev: 0
     nit: 1000
    njev: 0
  status: 0
 success: True
       x: array([4.49893948e-02, 4.43790738e-02, 4.08915328e-02, 8.60818913e+03,
       2.44622045e+04, 2.48735515e+04, 2.14801207e+04])
best ll: 91026.39509861523, bestParams: [0.11172657, 0.090561226, 0.053274393, 6340.836, 12428.718, 19741.955, 19129.791]
epoch 1
     fun: 90680.17847220492
 message: ['Maximum number of iteration reached']
    nfev: 18535
    nhev: 0
     nit: 1000
    njev: 0
  status: 0
 success: True
       x: array([4.87069860e-02, 4.87180474e-02, 4.18491463e-02, 9.10768397e+03,
       2.46671325e+04, 2.48377120e+04, 2.38725486e+04])
minPrevious 90678.94727806201
best ll: 90935.85000957745, bestParams: [0.049423933, 0.009214227, 0.066587955, 8144.4087, 14092.45, 19146.107, 19184

/Users/alexkotlar/projects/tada/mvl/likelihoods.py:257: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  altCountsFlat = tensor(altCountsFlat)


best ll: 91239.19500249799, bestParams: [0.07369001, 0.04880978, 0.06687576, 4858.3516, 16533.74, 12923.512, 14745.741]
epoch 0
     fun: 90861.78577540864
 message: ['Maximum number of iteration reached']
    nfev: 28688
    nhev: 0
     nit: 1000
    njev: 0
  status: 0
 success: True
       x: array([4.39126641e-02, 4.43361706e-02, 4.16877774e-02, 6.30811938e+03,
       1.80875402e+04, 1.85093776e+04, 1.54715766e+04])
best ll: 91471.4697084639, bestParams: [0.015408821, 0.025966937, 0.018776404, 6470.502, 16699.875, 22305.629, 7609.0425]
epoch 1
     fun: 90860.74902260769
 message: ['Maximum number of iteration reached']
    nfev: 21083
    nhev: 0
     nit: 1000
    njev: 0
  status: 0
 success: True
       x: array([4.31587597e-02, 4.55204559e-02, 4.21324394e-02, 8.65442240e+03,
       2.47671562e+04, 2.49529431e+04, 2.13176385e+04])
minPrevious 90861.78577540864
better by at >= 1; new ll:      fun: 90860.74902260769
 message: ['Maximum number of iteration reached']
    nfev: 210

...

/Users/alexkotlar/projects/tada/mvl/likelihoods.py:257: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  altCountsFlat = tensor(altCountsFlat)


best ll: 91243.91071417881, bestParams: [0.040498573, 0.039992083, 0.033642173, 6753.6836, 15692.065, 21096.775, 11753.506]
epoch 0
     fun: 91035.2144862661
 message: ['Maximum number of iteration reached']
    nfev: 21164
    nhev: 0
     nit: 1000
    njev: 0
  status: 0
 success: True
       x: array([4.56112050e-02, 5.15920592e-02, 4.19819102e-02, 8.88284249e+03,
       2.49907590e+04, 2.48726630e+04, 2.23069026e+04])
best ll: 91357.54566372334, bestParams: [0.040997185, 0.04618633, 0.0838608, 10846.453, 19839.768, 20632.074, 23618.115]
epoch 1
     fun: 91032.30772549672
 message: ['Maximum number of iteration reached']
    nfev: 26802
    nhev: 0
     nit: 1000
    njev: 0
  status: 0
 success: True
       x: array([4.40870682e-02, 4.61593746e-02, 4.11538469e-02, 8.28519028e+03,
       2.40384533e+04, 2.47662828e+04, 1.98040363e+04])
minPrevious 91035.2144862661
better by at >= 1; new ll:      fun: 91032.30772549672
 message: ['Maximum number of iteration reached']
    nfev: 26

/Users/alexkotlar/projects/tada/mvl/likelihoods.py:257: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  altCountsFlat = tensor(altCountsFlat)


best ll: 91167.15270193382, bestParams: [0.014420534, 0.012763254, 0.08490746, 10409.463, 14869.195, 12726.8955, 24758.049]
epoch 0
     fun: 90664.13815521165
 message: ['Maximum number of iteration reached']
    nfev: 18871
    nhev: 0
     nit: 1000
    njev: 0
  status: 0
 success: True
       x: array([4.50519321e-02, 4.80974897e-02, 4.37722990e-02, 9.33124432e+03,
       2.49952647e+04, 2.48946095e+04, 2.44922474e+04])
best ll: 91064.46507395245, bestParams: [0.13599314, 0.0092406655, 0.06631607, 5395.3354, 13053.832, 21606.041, 11194.925]
epoch 1
     fun: 90658.51054391943
 message: ['Maximum number of iteration reached']
    nfev: 21778
    nhev: 0
     nit: 1000
    njev: 0
  status: 0
 success: True
       x: array([3.97539028e-02, 3.85180081e-02, 4.21989798e-02, 5.29648514e+03,
       1.53948253e+04, 1.60815565e+04, 1.24519796e+04])
minPrevious 90664.13815521165
better by at >= 1; new ll:      fun: 90658.51054391943
 message: ['Maximum number of iteration reached']
    nfev

KeyboardInterrupt: 

In [29]:
import copy
resSim = {
        "allRes": None,
        "nEpochs": None,
        "bestRes": {
            "pis": None,
            "alphas": None,
            "PDV_c1true": None,
            "PDV_c2true": None,
            "PDV_cBothTrue": None,
            "PDV_c1inferred": None,
            "PDV_c2inferred": None,
            "PDV_cBothInferred": None,
        }
    }

cached6NormalSimResNonAnnealing = []

rrsSim = tensor([[2, 2, 1.5]])
pisSim = tensor([[.05, .05, .05]])

nCases = tensor([15e3, 15e3, 6e3])
nCtrls = tensor(5e5)
i = 0
for rrsSimRun in rrsSim:
    for pisSimRun in pisSim:
        afMeanRun = 1e-4
        generatingFn = genData.v6normal
        # In DSB:
        # 	No ID	ID	
        #         ASD+ADHD	684	217	
        #         ASD	3091	871	
        #         ADHD	3206	271	
        #         Control	5002	-	

        #         gnomAD	44779	(Non-Finnish Europeans in non-psychiatric exome subset)	

        #         Case total:	8340		
        #         Control total:	49781		
        # so we can use pDBoth = .1 * total_cases
        # needs tensor for shapes, otherwise "gamma_cpu not implemente for long", e.g rrShape=50.0 doesn't work...
        paramsRun = genData.genParams(rrMeans=rrsSimRun, pis=pisSimRun, afMean=afMeanRun, rrShape=tensor(50.), afShape=tensor(50.), nCases=nCases, nCtrls=nCtrls)[0]
        
        pDsRun = paramsRun["pDs"]
        pisRun = paramsRun["diseaseFractions"]
        print("params are:", paramsRun)
        
        cached6NormalSimResNonAnnealing.append({"params": paramsRun, "runs": []})
        for y in range(0, 10):
            start = time.time()
            r = generatingFn(**paramsRun)
            print("took", time.time() - start)
            
            resPointer = {
                **r,
                "generatingFn": generatingFn,
                "results": None,
            }

            cached6NormalSimResNonAnnealing[i]["runs"].append(resPointer)
            
            print(f"Run: {i}, {y}")
            
            xsRun = resPointer["altCounts"]
            afsRun = resPointer["afs"]
            affectedGenesRun = resPointer["affectedGenes"]
            unaffectedGenesRun = resPointer["unaffectedGenes"]

            runCostFnIdx = 16

            nEpochsRun = 10
            print("nEpochsRun", nEpochsRun)
            
            res = likelihoods.fitFnBivariate(xsRun, pDsRun, nEpochs=nEpochsRun, minLLThresholdCount=20, debug=True, costFnIdx=runCostFnIdx)
            bestRes = res["params"][-1]

            inferredPis = tensor(bestRes[0:3]) # 3-vector
            inferredAlphas = tensor(bestRes[3:]) # 4-vector, idx0 is P(!D|V)

            #### Calculate actual ###
            component1Afs = afsRun[affectedGenesRun[0]]
            c1true = (component1Afs / afMeanRun).mean(0)

            component2Afs = afsRun[affectedGenesRun[1]]
            c2true = (component2Afs / afMeanRun).mean(0)

            componentBothAfs = afsRun[affectedGenesRun[2]]
            cBothTrue = (componentBothAfs / afMeanRun).mean(0)

            ### calculate inferred values
            pds = tensor([1-pDsRun.sum(), *pDsRun])
            alphas = inferredAlphas.numpy()
            c1inferred = Dirichlet(tensor([alphas[0], alphas[1], alphas[0], alphas[2]]) * pds).sample([10_000]).mean(0)
            c2inferred = Dirichlet(tensor([alphas[0], alphas[0], alphas[2], alphas[2]]) * pds).sample([10_000]).mean(0)
            cBothInferred = Dirichlet(tensor([alphas[0], (alphas[1] + alphas[3]), (alphas[2] + alphas[3]), (alphas[1] + alphas[2] + alphas[3])]) * pds).sample([10_000]).mean(0)

            print(f"\n\nrun {i} results for rrs: {rrsSimRun}, pis: {pisSimRun}")
            print("Inferred pis:", inferredPis)
            print("\nP(D|V) true ans in component 1:", c1true)
            print("P(D|V) inferred in component 1:", c1inferred)
            print("\nP(D|V) true ans in component 1:", c2true)
            print("P(D|V) inferred in component both:", c2inferred)
            print("\nP(D|V) true ans in component both:", cBothTrue)
            print("P(D|V) inferred in component both:", cBothInferred,"\n\n")

            resToStore = copy.deepcopy(resSim)
            resToStore["allRes"] = res
            resToStore["nEpochs"] = nEpochsRun
            br = resToStore["bestRes"]
            br["pis"] = inferredPis
            br["alphas"] = inferredAlphas
            br["PDV_c1true"] = c1true
            br["PDV_c2true"] = c2true
            br["PDV_cBothTrue"] = cBothTrue
            br["PDV_c1inferred"] = c1inferred
            br["PDV_c2inferred"] = c2inferred
            br["PDV_cBothInferred"] = cBothInferred

            resPointer["results"] = resToStore
        
        i += 1

pDs are: tensor([0.0280, 0.0280, 0.0112])
params are: {'nGenes': 20000, 'nCases': tensor([15000., 15000.,  6000.]), 'nCtrls': tensor(500000.), 'pDs': tensor([0.0280, 0.0280, 0.0112]), 'diseaseFractions': tensor([0.0500, 0.0500, 0.0500]), 'rrShape': tensor(50.), 'rrMeans': tensor([2.0000, 2.0000, 1.5000]), 'afShape': tensor(50.), 'afMean': 0.0001}
TESTING WITH: nCases tensor([15000., 15000.,  6000.]) nCtrls tensor(500000.) rrMeans tensor([2.0000, 2.0000, 1.5000]) rrShape tensor(50.) afMean 0.0001 afShape tensor(50.) diseaseFractions tensor([0.0500, 0.0500, 0.0500]) pDs tensor([0.0280, 0.0280, 0.0112])
tensor([[2.0478, 2.3230, 4.9419],
        [1.1123, 3.1260, 6.4751],
        [3.7432, 4.8883, 4.0865],
        ...,
        [2.3440, 2.6389, 4.7504],
        [3.1413, 3.7973, 5.2277],
        [2.2387, 4.0079, 4.0649]], dtype=torch.float64)
startIndices [0, tensor(1000.), tensor(2000.)] endIndices tensor([1000., 2000., 3000.])
totalSamples 536000
took 8.850098848342896
Run: 0, 0
nEpochsRun 1

/Users/alexkotlar/projects/tada/mvl/likelihoods.py:257: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  altCountsFlat = tensor(altCountsFlat)


best ll: 91172.32504614866, bestParams: [0.013028687, 0.04237689, 0.055741724, 11929.878, 23005.342, 24812.355, 19184.58]
epoch 0
 final_simplex: (array([[1.44827568e-02, 4.20886696e-02, 5.23620101e-02, 8.85204497e+03,
        2.80864800e+04, 2.54391201e+04, 1.98356475e+04],
       [1.44827568e-02, 4.20886697e-02, 5.23620101e-02, 8.85204497e+03,
        2.80864800e+04, 2.54391201e+04, 1.98356475e+04],
       [1.44827568e-02, 4.20886695e-02, 5.23620101e-02, 8.85204496e+03,
        2.80864801e+04, 2.54391201e+04, 1.98356475e+04],
       [1.44827568e-02, 4.20886697e-02, 5.23620101e-02, 8.85204497e+03,
        2.80864800e+04, 2.54391201e+04, 1.98356475e+04],
       [1.44827568e-02, 4.20886695e-02, 5.23620101e-02, 8.85204496e+03,
        2.80864800e+04, 2.54391201e+04, 1.98356475e+04],
       [1.44827568e-02, 4.20886695e-02, 5.23620101e-02, 8.85204495e+03,
        2.80864801e+04, 2.54391201e+04, 1.98356475e+04],
       [1.44827568e-02, 4.20886695e-02, 5.23620101e-02, 8.85204495e+03,
       

/Users/alexkotlar/projects/tada/mvl/likelihoods.py:257: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  altCountsFlat = tensor(altCountsFlat)


best ll: 90741.90852394194, bestParams: [0.07122339, 0.0030399286, 0.054159407, 10150.92, 21346.87, 23431.73, 21332.012]
epoch 0
 final_simplex: (array([[6.75587190e-02, 3.35212914e-03, 5.38820010e-02, 8.95523095e+03,
        2.17844698e+04, 2.55800586e+04, 2.21139824e+04],
       [6.75587191e-02, 3.35212913e-03, 5.38820010e-02, 8.95523096e+03,
        2.17844698e+04, 2.55800586e+04, 2.21139824e+04],
       [6.75587191e-02, 3.35212913e-03, 5.38820010e-02, 8.95523095e+03,
        2.17844698e+04, 2.55800586e+04, 2.21139824e+04],
       [6.75587193e-02, 3.35212913e-03, 5.38820010e-02, 8.95523096e+03,
        2.17844698e+04, 2.55800585e+04, 2.21139824e+04],
       [6.75587193e-02, 3.35212913e-03, 5.38820010e-02, 8.95523096e+03,
        2.17844698e+04, 2.55800585e+04, 2.21139824e+04],
       [6.75587191e-02, 3.35212914e-03, 5.38820011e-02, 8.95523095e+03,
        2.17844697e+04, 2.55800586e+04, 2.21139824e+04],
       [6.75587194e-02, 3.35212913e-03, 5.38820010e-02, 8.95523096e+03,
        

...

/Users/alexkotlar/projects/tada/mvl/likelihoods.py:257: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  altCountsFlat = tensor(altCountsFlat)


best ll: 90866.38670202054, bestParams: [0.04227104, 0.043204933, 0.035604723, 4999.505, 6446.358, 10356.963, 18687.867]
epoch 0
 final_simplex: (array([[3.78586233e-02, 4.97323902e-02, 4.39350770e-02, 3.58313192e+03,
        1.04477019e+04, 1.02147190e+04, 8.49245920e+03],
       [3.78586235e-02, 4.97323902e-02, 4.39350773e-02, 3.58313192e+03,
        1.04477019e+04, 1.02147190e+04, 8.49245919e+03],
       [3.78586232e-02, 4.97323905e-02, 4.39350767e-02, 3.58313190e+03,
        1.04477020e+04, 1.02147190e+04, 8.49245911e+03],
       [3.78586227e-02, 4.97323902e-02, 4.39350774e-02, 3.58313190e+03,
        1.04477020e+04, 1.02147191e+04, 8.49245916e+03],
       [3.78586228e-02, 4.97323899e-02, 4.39350782e-02, 3.58313192e+03,
        1.04477019e+04, 1.02147191e+04, 8.49245925e+03],
       [3.78586223e-02, 4.97323901e-02, 4.39350782e-02, 3.58313191e+03,
        1.04477019e+04, 1.02147191e+04, 8.49245913e+03],
       [3.78586225e-02, 4.97323904e-02, 4.39350773e-02, 3.58313189e+03,
        

/Users/alexkotlar/projects/tada/mvl/likelihoods.py:257: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  altCountsFlat = tensor(altCountsFlat)


best ll: 90761.7781549094, bestParams: [0.04600825, 0.034627862, 0.03320431, 3705.6182, 7804.2446, 11532.189, 11376.845]
epoch 0
 final_simplex: (array([[4.79401002e-02, 3.59860707e-02, 4.02713565e-02, 3.34189612e+03,
        9.83545164e+03, 1.05043621e+04, 7.70126223e+03],
       [4.79401001e-02, 3.59860709e-02, 4.02713562e-02, 3.34189612e+03,
        9.83545164e+03, 1.05043621e+04, 7.70126223e+03],
       [4.79401002e-02, 3.59860711e-02, 4.02713562e-02, 3.34189612e+03,
        9.83545164e+03, 1.05043621e+04, 7.70126226e+03],
       [4.79401004e-02, 3.59860708e-02, 4.02713565e-02, 3.34189612e+03,
        9.83545164e+03, 1.05043620e+04, 7.70126228e+03],
       [4.79401003e-02, 3.59860708e-02, 4.02713564e-02, 3.34189612e+03,
        9.83545166e+03, 1.05043621e+04, 7.70126227e+03],
       [4.79401006e-02, 3.59860705e-02, 4.02713567e-02, 3.34189612e+03,
        9.83545165e+03, 1.05043620e+04, 7.70126230e+03],
       [4.79401005e-02, 3.59860707e-02, 4.02713564e-02, 3.34189612e+03,
        

KeyboardInterrupt: 

In [7]:
runSim()

pDs are: tensor([0.0280, 0.0280, 0.0112])
params are: {'nGenes': 20000, 'nCases': tensor([15000., 15000.,  6000.]), 'nCtrls': tensor(500000.), 'pDs': tensor([0.0280, 0.0280, 0.0112]), 'diseaseFractions': tensor([0.0500, 0.0500, 0.0500]), 'rrShape': tensor(50.), 'rrMeans': tensor([1.5000, 1.5000, 1.5000]), 'afShape': tensor(50.), 'afMean': 0.0001}
TESTING WITH: nCases tensor([15000., 15000.,  6000.]) nCtrls tensor(500000.) rrMeans tensor([1.5000, 1.5000, 1.5000]) rrShape tensor(50.) afMean 0.0001 afShape tensor(50.) diseaseFractions tensor([0.0500, 0.0500, 0.0500]) pDs tensor([0.0280, 0.0280, 0.0112])
tensor([[3.8293, 2.6289, 3.8956],
        [2.4995, 2.3651, 2.9228],
        [2.9986, 3.1782, 5.7793],
        ...,
        [1.6170, 2.4270, 3.9871],
        [2.6860, 2.5748, 3.5131],
        [3.5057, 4.4161, 3.9715]], dtype=torch.float64)
startIndices [0, tensor(1000.), tensor(2000.)] endIndices tensor([1000., 2000., 3000.])
totalSamples 536000
took 8.05718994140625
Run: 0, 0
nEpochsRun 10

/Users/alexkotlar/projects/tada/mvl/likelihoods.py:257: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  altCountsFlat = tensor(altCountsFlat)


best ll: 89254.57939981687, bestParams: [0.09598137, 0.18161891, 0.066081025, 8628.348, 19064.254, 11141.051, 24336.697]
epoch 0
     fun: 89087.15867372246
 message: ['Maximum number of iteration reached']
    nfev: 17512
    nhev: 0
     nit: 1000
    njev: 0
  status: 0
 success: True
       x: array([9.06655157e-01, 8.05196165e-03, 8.52313234e-02, 1.51882047e+04,
       1.59563997e+04, 2.49997166e+04, 2.49999993e+04])
took 263.1990089416504


run 0 results for rrs: tensor([1.5000, 1.5000, 1.5000]), pis: tensor([0.0500, 0.0500, 0.0500])
Inferred pis: tensor([0.9067, 0.0081, 0.0852], dtype=torch.float64)

P(D|V) true ans in component 1: tensor([0.8974, 0.0550, 0.0281, 0.0220], dtype=torch.float64)
P(D|V) inferred in component 1: tensor([0.9248, 0.0291, 0.0277, 0.0183], dtype=torch.float64)

P(D|V) true ans in component 1: tensor([0.8934, 0.0280, 0.0558, 0.0223], dtype=torch.float64)
P(D|V) inferred in component both: tensor([0.9098, 0.0273, 0.0449, 0.0180], dtype=torch.float64)

P(D|

/Users/alexkotlar/projects/tada/mvl/likelihoods.py:257: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  altCountsFlat = tensor(altCountsFlat)


best ll: 89545.34098332483, bestParams: [0.019650858, 0.005765289, 0.015805295, 8096.601, 15435.601, 23816.336, 12740.194]


KeyboardInterrupt: 

In [8]:
runSim(fitMethod='nelder-mead')

IN
pDs are: tensor([0.0280, 0.0280, 0.0112])
params are: {'nGenes': 20000, 'nCases': tensor([15000., 15000.,  6000.]), 'nCtrls': tensor(500000.), 'pDs': tensor([0.0280, 0.0280, 0.0112]), 'diseaseFractions': tensor([0.0500, 0.0500, 0.0500]), 'rrShape': tensor(50.), 'rrMeans': tensor([1.5000, 1.5000, 1.5000]), 'afShape': tensor(50.), 'afMean': 0.0001}
TESTING WITH: nCases tensor([15000., 15000.,  6000.]) nCtrls tensor(500000.) rrMeans tensor([1.5000, 1.5000, 1.5000]) rrShape tensor(50.) afMean 0.0001 afShape tensor(50.) diseaseFractions tensor([0.0500, 0.0500, 0.0500]) pDs tensor([0.0280, 0.0280, 0.0112])
tensor([[2.9334, 2.5262, 4.5972],
        [2.9295, 2.6827, 4.2659],
        [3.4033, 4.0980, 4.6098],
        ...,
        [3.2606, 1.8734, 3.4271],
        [2.4874, 2.7894, 4.5732],
        [3.8228, 2.9923, 4.2592]], dtype=torch.float64)
startIndices [0, tensor(1000.), tensor(2000.)] endIndices tensor([1000., 2000., 3000.])
totalSamples 536000
took 7.798516273498535
Run: 0, 0
nEpochsRu

/Users/alexkotlar/projects/tada/mvl/likelihoods.py:257: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  altCountsFlat = tensor(altCountsFlat)


best ll: 89504.27484045511, bestParams: [0.01460946, 0.02944035, 0.03863643, 3142.4583, 3569.245, 5062.629, 6662.2]
epoch 0
 final_simplex: (array([[1.98604401e-02, 3.41540281e-02, 3.55878743e-02, 1.65112738e+03,
        5.21052634e+03, 4.99263541e+03, 2.62354673e+03],
       [1.98604401e-02, 3.41540280e-02, 3.55878742e-02, 1.65112738e+03,
        5.21052633e+03, 4.99263546e+03, 2.62354667e+03],
       [1.98604401e-02, 3.41540281e-02, 3.55878743e-02, 1.65112737e+03,
        5.21052636e+03, 4.99263542e+03, 2.62354672e+03],
       [1.98604400e-02, 3.41540281e-02, 3.55878741e-02, 1.65112738e+03,
        5.21052634e+03, 4.99263542e+03, 2.62354670e+03],
       [1.98604401e-02, 3.41540283e-02, 3.55878742e-02, 1.65112737e+03,
        5.21052638e+03, 4.99263537e+03, 2.62354673e+03],
       [1.98604401e-02, 3.41540280e-02, 3.55878744e-02, 1.65112737e+03,
        5.21052634e+03, 4.99263546e+03, 2.62354667e+03],
       [1.98604400e-02, 3.41540281e-02, 3.55878747e-02, 1.65112738e+03,
        5.210

KeyboardInterrupt: 

In [26]:
res = np.load("./mvln-sim-mvln.npy", allow_pickle=True)


KeyboardInterrupt: 

In [ ]:
resByParams = []
for runSet in res:
    params = (runSet["params"]["diseaseFractions"], runSet["params"]["rrMeans"], runSet["params"]["rrShape"])
#     print("param", params)
    res = []
#     if params not in resByParam:
#         resByParams[params] = []
    
    for run in runSet["runs"]:
        if run is None or "results" not in run or run["results"] is None:
            print(f"no results found for {params}")
            continue
        res.append(run["results"])
    resByParams.append([params, res])

np.save("mvln-sim-mvln-results2", resByParams)


In [27]:
resByParams = np.load("mvln-sim-mvln-results.npy", allow_pickle=True)


In [28]:
resByParams

array([[(tensor([0.0500, 0.0500, 0.0500]), tensor([2.0000, 2.0000, 1.5000]), tensor(50.)),
        list([{'allRes': {'lls': [90916.4783589971, 90708.31368424412, 90689.04874179952, 90677.23983613223], 'params': [array([3.87333861e-02, 1.40812486e-01, 7.59025824e-02, 9.31141067e+03,
       2.18500134e+04, 1.68652588e+04, 2.63598593e+04]), array([1.67543257e-02, 3.13060638e-02, 4.29277051e-02, 2.50496081e+03,
       8.64518025e+03, 8.37694873e+03, 4.74332877e+03]), array([4.07980539e-02, 5.92690249e-02, 4.06981218e-02, 4.76417261e+03,
       1.30584069e+04, 1.28133394e+04, 1.26034808e+04]), array([3.56317955e-02, 3.20989858e-02, 4.08777915e-02, 5.02625412e+03,
       1.52388589e+04, 1.62573997e+04, 1.10763793e+04])], 'llTrajectory': [90916.4783589971, 90708.31368424412, 90727.26669070916, 90689.04874179952, 90677.23983613223, 90676.96371994552, 90738.01226968745, 90784.31279984681, 90787.42281897092, 90677.4934384852]}, 'nEpochs': 10, 'bestRes': {'pis': tensor([0.0356, 0.0321, 0.0409], d

In [9]:

# plt.subplot([1,len(resByParams)])
i = 0
with open("mvln-sim-mvln-res.tsv", "w") as file:
    file.write(f"Notes: 15000 samples1, 15000 samples2, 6000 samplesBoth; rrs generated from normal distribution with. 1 variance, .4 covariance, and individual effect rr summed with shared-effect rr in genes affecting both conditions\n")
    file.write(f"\tmean\tstd\n")
    for res in resByParams:
        i += 1

        paramsRun = res[0]
        resRun = res[1]

        pis = tensor([x["bestRes"]["pis"].numpy() for x in resRun])
        PDV_c1true = tensor([x["bestRes"]["PDV_c1true"].numpy() for x in resRun])
        PDV_c2true = tensor([x["bestRes"]["PDV_c2true"].numpy() for x in resRun])
        PDV_c3true = tensor([x["bestRes"]["PDV_cBothTrue"].numpy() for x in resRun])
        PDV_c1inferred = tensor([x["bestRes"]["PDV_c1inferred"].numpy() for x in resRun])
        PDV_c2inferred = tensor([x["bestRes"]["PDV_c2inferred"].numpy() for x in resRun])
        PDV_c3inferred = tensor([x["bestRes"]["PDV_cBothInferred"].numpy() for x in resRun])

        file.write(f"\n\ntrue params: \t{paramsRun} \n\n")

        file.write(f"pi\t {pis.mean(0).numpy()} \t  {pis.std(0).numpy()} \n")

        file.write(f"PDV_c1inferred \t {PDV_c1inferred.mean(0).numpy()}\t {PDV_c1inferred.std(0).numpy()}\n")
        file.write(f"PDV_c1true \t {PDV_c1true.mean(0).numpy()} \t {PDV_c1true.std(0).numpy()}\n")
        file.write(f"PDV_c2inferred \t {PDV_c2inferred.mean(0).numpy()} \t {PDV_c2inferred.std(0).numpy()}\n")
        
        file.write(f"PDV_c2true \t {PDV_c2true.mean(0).numpy()} \t {PDV_c2true.std(0).numpy()}\n")
        
        file.write(f"PDV_c3inferred \t {PDV_c3inferred.mean(0).numpy()} \t {PDV_c3inferred.std(0).numpy()}\n")
        
        file.write(f"PDV_c3true \t {PDV_c3true.mean(0).numpy()} \t {PDV_c3true.std(0).numpy()}\n")

    #     plt.figure(i)
    #     plt.plot(t, s1)
    #     plt.plot(t, 2*s1)
        # plt.subplot(222)
        # plt.plot(t, 2*s1)

In [16]:
with open('./mvln-sim-mvn.json', 'w') as outfile:
    json.dump(cached6NormalSimRes, outfile)

TypeError: Object of type Tensor is not JSON serializable

In [22]:
def one(a, b, c, d = 5):
    print(a, b, c, d)
    
def two(*args, **kwargs):
    one(*args, **kwargs)
    
two(1, 2, c = 3)

1 2 3 5


In [ ]:
pool_size = multiprocessing.cpu_count()
os.system('taskset -cp 0-%d %s' % (pool_size, os.getpid()))

genData.runSim(fitMethod="nelder-mead", mt=True)

In [1]:
import os
os.system('taskset -p %s' %os.getpid())

32512

In [1]:
import multiprocessing
import numpy as np
import math
import time
import os

def compute_something(t):
    a = 0.
    for i in range(10000000):
        a = math.sqrt(t)
    return a




print("Pool size:", pool_size)
pool = multiprocessing.Pool(processes=pool_size)

inputs = range(10)

tic = time.time()
builtin_outputs = map(compute_something, inputs)
print('Built-in:', time.time() - tic)

tic = time.time()
pool_outputs = pool.map(compute_something, inputs)
print('Pool    :', time.time() - tic)

Pool size: 12
Built-in: 0.00014591217041015625
Pool    : 1.7946691513061523


Process ForkPoolWorker-10:
Process ForkPoolWorker-8:
Process ForkPoolWorker-3:
Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Process ForkPoolWorker-9:
Process ForkPoolWorker-4:
Process ForkPoolWorker-12:
Process ForkPoolWorker-11:
Process ForkPoolWorker-2:
Process ForkPoolWorker-7:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/alexkotlar/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/alexkotlar/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/alexkotlar/miniconda3/lib/python3.7/multiproces

In [28]:
# import torch 
# import pyro
# from torch.multiprocessing import Process
# from torch.distributions import Uniform
# import time
# import numpy as np
# def writer():
#     message = f"I am Process {i}"
#     print(torch.utils.data.get_worker_info())
#     x = np.random.seed()
#     x = np.random.randint(1000)
#     print('seed', x)
#     torch.manual_seed(x)
#     pis = Uniform(1/100, .5).rsample([3])
#     print("r is", pis)
#     time.sleep(1)

# def reader(i,q):
#     message = q.get()
#     print("got message", message)

# for i in range(10):
   
#     Process(target=writer, args=()).start()
#     time.sleep(.5)
#    # Create multiprocessing pool
#     p = Pool(10)
#    # Create a group of parallel readers and start them
#    # Number of readers is matching the number of writers
#    # However, the number of simultaneously running
#    # readers is constrained to the pool size
#     readers = []
#     for i in range(10):
#         readers.append(p.apply_async(reader, (i,q,)))
#     # Wait for the asynchrounous reader threads to finish
#     [r.get() for r in readers]
    
    
#     # Establish communication queues
#     tasks = torch.multiprocessing.JoinableQueue()
#     results = torch.multiprocessing.Queue()
    
#     # Start consumers
#     num_consumers = torch.multiprocessing.cpu_count()
#     print('Creating %d consumers' % num_consumers)
#     consumers = [ Consumer(tasks, results)
#                   for i in range(num_consumers) ]
#     for w in consumers:
#         w.start()
    
#     # Enqueue jobs
#     for i in range(nEpochs):
#         tasks.put(Task(altCountsByGene, pDs, 1, minLLThresholdCount, K, debug, costFnIdx, method))
    
#     # Add a poison pill for each consumer
#     for i in range(nEpochs):
#         tasks.put(None)

#     # Wait for all of the tasks to finish
#     tasks.join()
    
#     # Start printing results
#     while nEpochs:
#         result = results.get()
#         print('Result:', result)
#         num_jobs -= 1


None
seed 426
r is tensor([0.0901, 0.3060, 0.4284])
None
seed 549
r is tensor([0.1975, 0.4509, 0.1906])
None
seed 47
r is tensor([0.0360, 0.0345, 0.2392])
None
seed 992
r is tensor([0.2402, 0.4320, 0.4001])
None
seed 419
r is tensor([0.1488, 0.4900, 0.2033])
None
seed 62
r is tensor([0.3242, 0.4162, 0.1136])
None
seed 945
r is tensor([0.1849, 0.2026, 0.3619])
None
seed 530
r is tensor([0.1196, 0.0993, 0.4660])
None
seed 791
r is tensor([0.1833, 0.3470, 0.0177])
None
seed 30
r is tensor([0.4514, 0.3757, 0.2411])


In [29]:






import os
from collections import defaultdict
import torch
import numpy as np
import scipy.stats
from torch.distributions import constraints
from matplotlib import pyplot
%matplotlib inline

import pyro
import pyro.distributions as dist
from pyro import poutine
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam
from pyro.infer import SVI, TraceEnum_ELBO, config_enumerate, infer_discrete

smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.3.0')
pyro.enable_validation(True)

AssertionError: 

In [123]:
genData.runSim(fitMethod="nelder-mead", mt=True)

MT is  True
pDs are: tensor([0.0446, 0.0446, 0.0179])
params are: {'nGenes': 20000, 'nCases': tensor([15000., 15000.,  6000.]), 'nCtrls': tensor(300000.), 'pDs': tensor([0.0446, 0.0446, 0.0179]), 'diseaseFractions': tensor([0.0500, 0.0500, 0.0500]), 'rrShape': tensor(50.), 'rrMeans': tensor([1.5000, 1.5000, 1.5000]), 'afShape': tensor(50.), 'afMean': 0.0001}
TESTING WITH: nCases tensor([15000., 15000.,  6000.]) nCtrls tensor(300000.) rrMeans tensor([1.5000, 1.5000, 1.5000]) rrShape tensor(50.) afMean 0.0001 afShape tensor(50.) diseaseFractions tensor([0.0500, 0.0500, 0.0500]) pDs tensor([0.0446, 0.0446, 0.0179])


test tensor 1.0,0.4,0.4,0.4,1.0,0.4,0.4,0.4,1.0
tensor([[4.0103, 4.1868, 6.2500],
        [3.3129, 4.4713, 5.1891],
        [2.3431, 3.4354, 5.1669],
        ...,
        [2.1873, 2.7176, 4.4710],
        [3.5895, 2.7138, 3.8816],
        [3.3634, 3.6205, 4.9512]], dtype=torch.float64)
startIndices [0, tensor(1000.), tensor(2000.)] endIndices tensor([1000., 2000., 3000.])
tot

Process ForkPoolWorker-225:
Process ForkPoolWorker-227:
Process ForkPoolWorker-231:
Process ForkPoolWorker-233:
Process ForkPoolWorker-228:
Process ForkPoolWorker-226:
Process ForkPoolWorker-234:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/alexkotlar/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-224:
  File "/Users/alexkotlar/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/alexkotlar/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/alexkotlar/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/alexkotlar/minico

KeyboardInterrupt: 

In [49]:
from pyro.distributions import DirichletMultinomial

In [50]:
DirichletMultinomial(total_count=1000, concentration=[1, 1, 1, 1]).log_prob([10, 10, 10, 10.])

AttributeError: 'list' object has no attribute 'dtype'

In [66]:
rrs = tensor([1.5, 1.5, 1.5])
pis = tensor([.05, .05, .05])
nCases = tensor([15e3, 15e3, 6e3])
nCtrls = tensor(3e5)
afMean = 1e-4
rrShape=tensor(50.)
afShape=tensor(50.)
generatingFn =  genData.v6normal
fitMethod = 'nelder-mead'
nEpochs=20
mt = True
covShared=tensor([[1,.4,.4], [.4, 1, .4], [.4, .4, 1]])
covSingle=tensor([[1, 0], [0, 1]])

params = genData.genParams(rrMeans=rrs, pis=pis, afMean=afMean, rrShape=rrShape, afShape=afShape, nCases=nCases, nCtrls=nCtrls)[0]
testData = generatingFn(**params, covShared=covShared, covSingle=covSingle)

pDs are: tensor([0.0446, 0.0446, 0.0179])
TESTING WITH: nCases tensor([15000., 15000.,  6000.]) nCtrls tensor(300000.) rrMeans tensor([1.5000, 1.5000, 1.5000]) rrShape tensor(50.) afMean 0.0001 afShape tensor(50.) diseaseFractions tensor([0.0500, 0.0500, 0.0500]) pDs tensor([0.0446, 0.0446, 0.0179])


test tensor 1.0,0.4,0.4,0.4,1.0,0.4,0.4,0.4,1.0
tensor([[4.3004, 4.5279, 4.6280],
        [2.8897, 3.4723, 5.9054],
        [2.6317, 3.3074, 5.0648],
        ...,
        [2.2534, 3.4058, 4.8345],
        [2.9521, 2.5474, 4.0645],
        [2.7626, 2.7038, 5.4264]], dtype=torch.float64)
startIndices [0, tensor(1000.), tensor(2000.)] endIndices tensor([1000., 2000., 3000.])
totalSamples 336000


In [ ]:
testData

In [69]:
params

{'nGenes': 20000,
 'nCases': tensor([15000., 15000.,  6000.]),
 'nCtrls': tensor(300000.),
 'pDs': tensor([0.0446, 0.0446, 0.0179]),
 'diseaseFractions': tensor([0.0500, 0.0500, 0.0500]),
 'rrShape': tensor(50.),
 'rrMeans': tensor([1.5000, 1.5000, 1.5000]),
 'afShape': tensor(50.),
 'afMean': 0.0001}

In [63]:
params = likelihoods.fitFnBivariateMT(params["altCount"])

Signature:
likelihoods.fitFnBivariateMT(
    altCountsByGene,
    pDs,
    nEpochs=20,
    minLLThresholdCount=100,
    K=4,
    debug=False,
    costFnIdx=0,
    method='nelder-mead',
)
Docstring: <no docstring>
File:      ~/projects/tada/mvl/likelihoods.py
Type:      function


In [70]:
testFit = likelihoods.fitFnBivariateMT(testData["altCounts"], params["pDs"], nEpochs=12)

pd1, pd2, pdBoth, pdCtrl:  tensor([0.8929, 0.0446, 0.0446, 0.0179])
pd1, pd2, pdBoth, pdCtrl:  tensor([0.8929, 0.0446, 0.0446, 0.0179])
pd1, pd2, pdBoth, pdCtrl:  tensor([0.8929, 0.0446, 0.0446, 0.0179])
pd1, pd2, pdBoth, pdCtrl:  tensor([0.8929, 0.0446, 0.0446, 0.0179])
pd1, pd2, pdBoth, pdCtrl:  tensor([0.8929, 0.0446, 0.0446, 0.0179])
pd1, pd2, pdBoth, pdCtrl:  tensor([0.8929, 0.0446, 0.0446, 0.0179])
pd1, pd2, pdBoth, pdCtrl:  tensor([0.8929, 0.0446, 0.0446, 0.0179])
pd1, pd2, pdBoth, pdCtrl:  tensor([0.8929, 0.0446, 0.0446, 0.0179])
pd1, pd2, pdBoth, pdCtrl:  tensor([0.8929, 0.0446, 0.0446, 0.0179])
pd1, pd2, pdBoth, pdCtrl:  tensor([0.8929, 0.0446, 0.0446, 0.0179])
pd1, pd2, pdBoth, pdCtrl:  tensor([0.8929, 0.0446, 0.0446, 0.0179])
pd1, pd2, pdBoth, pdCtrl:  tensor([0.8929, 0.0446, 0.0446, 0.0179])
method nelder-mead costFn <function likelihoodBivariateFast.<locals>.jointLikelihood at 0x7ff398b02a70>
method nelder-mead costFn <function likelihoodBivariateFast.<locals>.jointLikeli

In [72]:
bestLL = None
bestParams = None
for x in testFit:
    if bestLL is None or x["lls"][0] < bestLL:
        bestParams = x["params"][0]
        bestLL = x["lls"][0]
    

In [73]:
bestLL

88658.88416279141

In [84]:
bestParams = tensor(bestParams)


/Users/alexkotlar/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


tensor([4.0687e-02, 4.4221e-02, 4.1726e-02, 1.5900e+02, 4.0201e+02, 3.9009e+02,
        3.7284e+02], dtype=torch.float64)

In [85]:
bestParams

tensor([4.0687e-02, 4.4221e-02, 4.1726e-02, 1.5900e+02, 4.0201e+02, 3.9009e+02,
        3.7284e+02], dtype=torch.float64)

In [75]:
testCounts = testData["altCounts"]

In [95]:
testUnaffectedGenes = testData["unaffectedGenes"]
testAffectedGenes = testData["affectedGenes"]
testAllPDs = tensor([1-params["pDs"].sum(), *params["pDs"]])

In [110]:
# bayesFactor(n, altCount,pDs,rrMeans,rrsShape,afMeans,afShape,alpha0, alpha1, alpha2, alphaBoth):
bfsUnaffected = []
for unaffectedGene in testUnaffectedGenes:
    testCount = testCounts[unaffectedGene]
    bfsUnaffected.append(bayes.bayesFactor(testCount.sum(), testCount, testAllPDs, bestParams[0:3], bestParams[3:]))
bfsUnaffected = tensor(bfsUnaffected)

In [112]:
bfsUnaffected

tensor([1.0125e-01, 8.5311e-06, 1.6218e-04,  ..., 3.2103e-04, 1.1671e-04,
        5.6422e-03], dtype=torch.float64)

NameError: name 'bfsUnaffected' is not defined

In [ ]:
# bayesFactor(n, altCount,pDs,rrMeans,rrsShape,afMeans,afShape,alpha0, alpha1, alpha2, alphaBoth):
bfsAffected1 = []
for affectedGene in testAffectedGenes[0]:
    print(affectedGene, "count:", testCounts[affectedGene])
    testCount = testCounts[affectedGene]
    bfsAffected.append(bayes.bayesFactor(testCount.sum(), testCount, testAllPDs, bestParams[0:3], bestParams[3:]))
bfsAffected1 = tensor(bfsAffected1)

In [122]:
# bayesFactor(n, altCount,pDs,rrMeans,rrsShape,afMeans,afShape,alpha0, alpha1, alpha2, alphaBoth):
bfsAffected2 = []
for affectedGene in testAffectedGenes[1]:
    print(affectedGene, "count:", testCounts[affectedGene])
    testCount = testCounts[affectedGene]
    bfsAffected2.append(bayes.bayesFactor(testCount.sum(), testCount, testAllPDs, bestParams[0:3], bestParams[3:]))
bfsAffected2 = tensor(bfsAffected2)

1000 count: tensor([27.,  5.,  2.,  1.], dtype=torch.float64)
bfs are:  8.507990624958355 0.9929259704437408 1.1011605754837361 max is bf1
1001 count: tensor([27.,  1.,  3.,  2.], dtype=torch.float64)
bfs are:  0.7831355588613914 3.9680669560560147 0.16022523954973203 max is bf2
1002 count: tensor([24.,  0.,  5.,  0.], dtype=torch.float64)
bfs are:  0.15970660319136384 8.322358666428165 0.0740468383501618 max is bf2
1003 count: tensor([21.,  1.,  4.,  2.], dtype=torch.float64)
bfs are:  1.55250387085329 15.614736793822905 4.009810419576124 max is bf2
1004 count: tensor([28.,  0.,  0.,  2.], dtype=torch.float64)
bfs are:  0.5417786248275871 0.5766991103987299 0.0014611549488473359 max is bf2
1005 count: tensor([27.,  1.,  4.,  1.], dtype=torch.float64)
bfs are:  0.386773588064856 4.06769416253703 0.10893903805115321 max is bf2
1006 count: tensor([21.,  0.,  0.,  0.], dtype=torch.float64)
bfs are:  0.18938562889156788 0.20622900772462505 0.0006421326351400891 max is bf2
1007 count: tenso

KeyboardInterrupt: 

In [ ]:
bfsAffected